In [ ]:
!pip install deep-sort-realtime opencv-python ultralytics

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython.display import Video

In [ ]:
model = YOLO('yolov8n.pt')
tracker = DeepSort(max_age=30)

In [ ]:
cap = cv2.VideoCapture("/content/traffic_2.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("output_fixed.avi", cv2.VideoWriter_fourcc(*"XVID"), fps, (w, h))

In [ ]:
prev_pos, speed_hist = {}, {}

CONF_THRESH = 0.45
p_far = 0.06
p_near = 0.02
SMOOTH_FRAMES = 5

In [ ]:
def mpp(y):
    y = np.clip(y, 0, h)
    return p_far + (p_near - p_far) * (y / h)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    res = model(frame, verbose=False)
    boxes = res[0].boxes.xyxy.cpu().numpy()
    cls = res[0].boxes.cls.cpu().numpy().astype(int)
    conf = res[0].boxes.conf.cpu().numpy()

    dets = []
    for b, c, cf in zip(boxes, cls, conf):
        if cf < CONF_THRESH or c not in [2,3,5,7]:
            continue
        x1,y1,x2,y2 = map(int,b)
        dets.append([[x1,y1,x2-x1,y2-y1], float(cf), "veh"])

    tracks = tracker.update_tracks(dets, frame=frame)
    for t in tracks:
        if not t.is_confirmed():
            continue
        tid = t.track_id
        x1,y1,x2,y2 = map(int,t.to_ltrb())
        cx,cy = (x1+x2)//2,(y1+y2)//2
        spd=0
        if tid in prev_pos:
            px,py = prev_pos[tid]
            dp = np.hypot(cx-px, cy-py)
            s1,s2 = mpp(cy), mpp(py)
            spd = dp * ((s1+s2)/2) * fps * 3.6
            hist = speed_hist.get(tid,[])
            hist.append(spd)
            if len(hist)>SMOOTH_FRAMES: hist.pop(0)
            speed_hist[tid]=hist
            spd=np.mean(hist)
        prev_pos[tid]=(cx,cy)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(20,160,255),2)
        cv2.putText(frame,f"ID {tid}|{spd:.1f} km/h",(x1,y1-8),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
    out.write(frame)


In [ ]:
cap.release()
out.release()

In [ ]:
s = cv2.VideoCapture("output_fixed.avi")
s.set(cv2.CAP_PROP_POS_FRAMES, fps*2)
r,fr = s.read()
if r: cv2_imshow(fr)
s.release()
Video("output_fixed.avi", embed=True)